<a href="https://colab.research.google.com/github/Rachna0604/Devrev/blob/main/FinalDevrev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk==3.4.5
!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 30.8 MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449924 sha256=f69a1958a8a4ca84643805a23096acddb3acbb5dbbf75bda9e16e88a97f493e5
  Stored in directory: /root/.cache/pip/wheels/23/18/48/8fd6ec11da38406b309470566d6f099c04805d2ec61d7829e7
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 9.3 MB/s 
     |████████████████████████████████| 7.6 MB 45.4 MB/s 
     |████████████████████████████████| 182 kB 65.6 MB/s 


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
#for pre processing
import csv,numpy 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# accessing g drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#1

Predicting paragraphs from question and theme

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def predict_answer(question, paragraphs, theme):
    # Filter the paragraphs to only include those with the correct theme
    theme_paragraphs = [p['text'] for p in paragraphs if p['theme'] == theme]
    
    # Create a TfidfVectorizer to convert the paragraphs into numerical vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(theme_paragraphs)
    
    # Convert the question into a numerical vector
    question_vector = vectorizer.transform([question])
    
    # Calculate the cosine similarity between the question and each paragraph
    similarity = cosine_similarity(question_vector, X)
    
    # Find the paragraph with the highest similarity score
    max_similarity_index = similarity.argmax()
    max_similarity_score = similarity[0, max_similarity_index]
    
    # If the maximum similarity score is above a certain threshold, return the paragraph
    if max_similarity_score > 0.05:
        return theme_paragraphs[max_similarity_index]
    else:
        return None


In [ ]:
question = "What is the capital of France?"
paragraphs = [
    {'text': 'Paris is the capital of France', 'theme': 'Geography'},
    {'text': 'Lyon is a city in France', 'theme': 'Geography'},
    {'text': 'The Pythagorean theorem states that a^2 + b^2 = c^2', 'theme': 'Mathematics'},
]

# Predict the answer to the question
answer = predict_answer(question, paragraphs, 'Geography')

# Print the answer
if answer:
    print(answer)
else:
    print("No suitable paragraph found")

Paris is the capital of France


Saving the paragraphs,questions and themes in the way it can be passed through the function predict_answer

In [ ]:
path = '/content/drive/MyDrive/DevRev/train_data.csv'
data = pd.read_csv(path)
Paragraphs = []
for para in data['Paragraph']:
    if para not in Paragraphs:
        Paragraphs.append(para)
questions = []
for q in data['Question']:
    if q not in questions:
          questions.append(q)
themes = []
for t in data['Theme']:
    if t not in themes:
        themes.append(t)
PT = []
Themes = []
for element in Paragraphs:
    Themes.append(data['Theme'][data.loc[data['Paragraph'] == element]['Theme'].index[0]])
for i in range(len(Themes)):
    PT.append({'text': Paragraphs[i], 'theme': Themes[i]})
question = questions[0]
answer = predict_answer(question, PT, 'Beyoncé')
if answer:
    print(answer)
else:
    print("No suitable paragraph found")

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


#2

Traning a model    
**this has no continuation with the above code , it's a seperate method tried 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from the CSV file
df = pd.read_csv("/content/drive/MyDrive/DevRev/train_data.csv")

# Preprocess the data
df["question"] = df["Question"].str.lower()
df["paragraph"] = df["Paragraph"].str.lower()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[["question", "paragraph"]], df["Theme"], test_size=0.2)

# Vectorize the data using a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train["question"])
X_test_vectors = vectorizer.transform(X_test["question"])

# Train a support vector machine (SVM) classifier
clf = SVC()
clf.fit(X_train_vectors, y_train)

# Predict the theme labels for the test data
y_pred = clf.predict(X_test_vectors)


# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Calculate the precision of the model
precision = precision_score(y_test, y_pred, average='macro')

# Calculate the recall of the model
recall = recall_score(y_test, y_pred, average='macro')

# Calculate the F1 score of the model
f1 = f1_score(y_test, y_pred, average='macro')

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.622876557191393
Precision: 0.7600728344154949
Recall: 0.5839213487139984
F1 Score: 0.6357891846760749


In [1]:
# Preprocess the new question
new_question = "Who was Hayek's father?"
new_question = new_question.lower()

# Vectorize the new question
new_question_vector = vectorizer.transform([new_question])

# Use the model to make a prediction
prediction = clf.predict(new_question_vector)[0]

# Print the prediction
print("Prediction:", prediction)

NameError: ignored

#3

Below is another method of approaching the question which I tried earlier, it's not completely working, but approach can be referred

In [ ]:
def predicting_answer(question, paragraphs):
  #pre process
  question_tokens = preprocess_text(question)
  paragraphs_tokens = [preprocess_text(p) for p in paragraphs]
  #generate embeddings
  question_embeddings = nlp_model(question_tokens)
  paragraphs_embeddings = [nlp_model(p) for p in paragraphs_tokens]
  #compare embeddings i.e. similarity measure
  most_similar_paragraph = find_most_similar(question_embeddings, paragraphs_embeddings)
  #threshold
  if most_similar_paragraph[1] > SIMILARITY_THRESHOLD:
    return paragraphs[most_similar_paragraph[0]]
  else:
    return None

Pre processing

In [ ]:
# column index of the column we want to read
column_index = 2

# list to store the column values
column_values = []

# open the CSV file in read mode
'''
with open('/content/drive/MyDrive/DevRev/train_data.csv', 'r') as csv_file:
    # creating a CSV reader object
    reader = csv.reader(csv_file)
    
    # iterating over the rows in the CSV file
    for row in reader:
        # appending the value at the specified column index to the list
        column_values.append(row[column_index])
'''
path = '/content/drive/MyDrive/DevRev/train_data.csv'
data = pd.read_csv(path)
for para in data['Paragraph']:
  column_values.append(para)

# column_values now contains the values from the specified column

# list of paragraphs
paragraphs = column_values

# question
question = "What is the purpose of preprocessing text data?"

# lowercase the paragraphs and question
paragraphs = [p.lower() for p in paragraphs]
question = question.lower()

# removing stop words and punctuation from the paragraphs and question
stop_words = set(stopwords.words('english'))
punctuation = set(['.', ',', ':', ';', '?'])

filtered_paragraphs = []
for p in paragraphs:
    filtered_p = []
    for w in word_tokenize(p):
        if w not in stop_words and w not in punctuation:
            filtered_p.append(w)
    filtered_paragraphs.append(filtered_p)

filtered_question = []
for w in word_tokenize(question):
    if w not in stop_words and w not in punctuation:
        filtered_question.append(w)

# stem the paragraphs and question
stemmer = PorterStemmer()

stemmed_paragraphs = []
for p in filtered_paragraphs:
    stemmed_p = []
    for w in p:
        stemmed_p.append(stemmer.stem(w))
    stemmed_paragraphs.append(stemmed_p)

stemmed_question = []
for w in filtered_question:
    stemmed_question.append(stemmer.stem(w))

# create a list of tokens for the paragraphs and question
tokens_p = []
tokens_q = []
for p in stemmed_paragraphs:
    tokens_p.append(p)
tokens_q.extend(stemmed_question)
print(tokens_q)

['purpos', 'preprocess', 'text', 'data']


In [ ]:
def preprocess_text(text):
  # Tokenize the text
  tokens = nltk.word_tokenize(text)
  
  # Stem the tokens
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  
  return stemmed_tokens

# Pre-process the question
question = "What is the capital of France?"
question_tokens = preprocess_text(question)
print(question_tokens)  

# Pre-process a paragraph
paragraph = "Paris is the capital of France. It is located in the northern central part of the country and is home to many famous landmarks, including the Eiffel Tower."
paragraph_tokens = preprocess_text(paragraph)
print(paragraph_tokens)  


['what', 'is', 'the', 'capit', 'of', 'franc', '?']
['pari', 'is', 'the', 'capit', 'of', 'franc', '.', 'It', 'is', 'locat', 'in', 'the', 'northern', 'central', 'part', 'of', 'the', 'countri', 'and', 'is', 'home', 'to', 'mani', 'famou', 'landmark', ',', 'includ', 'the', 'eiffel', 'tower', '.']


In [ ]:
from transformers import BertModel, BertTokenizer
# Initialize the BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def generate_embeddings(tokens):
  # Encode the text
  input_ids = tokenizer(tokens, return_tensors='pt',padding=True,truncation=True)
  #the output is input tensors in the form of a PyTorch tensor
  print(input_ids)
# Generate embeddings for the question tokens
question_embeddings = generate_embeddings(question_tokens)

# Generate embeddings for the paragraph tokens
paragraph_embeddings = generate_embeddings(paragraph_tokens)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  2054,   102,     0],
        [  101,  2003,   102,     0],
        [  101,  1996,   102,     0],
        [  101,  6178,  4183,   102],
        [  101,  1997,   102,     0],
        [  101, 23151,  2278,   102],
        [  101,  1029,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 0]])}
{'input_ids': tensor([[  101, 11968,  2072,   102,     0],
        [  101,  2003,   102,     0,     0],
        [  101,  1996,   102,     0,     0],
        [  101,  6178,  4183,   102,     0],
        [  101,  1997,   102,     0,     0],
        [  101, 23151,  2278,   102,     0],
        [  101,  1012,   102,     0,     0],
        [  101,  2009,   102,     0,     

In [ ]:
from scipy import spatial

def cosine_similarity(emb1, emb2):
    similarity = 1 - spatial.distance.cosine(emb1, emb2)
    return similarity

#note
#in this we have to give the axes so for each paragraph in the func defined
#so the word embeddings should be stored in a list, that list hould be passed the function
#like below given hashtags
#paragraph_embeddings = [word_embedding_1, word_embedding_2, word_embedding_3]
#question_embeddings = [word_embedding_4, word_embedding_5, word_embedding_6]

# Calculating the cosine similarity between the two paragraphs
similarity = cosine_similarity(paragraph_embeddings, question_embeddings)

# Determining which paragraph is more similar based on the similarity value
if similarity > 0.8:
    print(paragraph)
else:
    print("Paragraph is not similar")


In [ ]:
#note paragraphs embeddings means there are a lot of paragraphs' embeddings *****
def find_most_similar(question_embeddings, paragraphs_embeddings):
  max_similarity = -1
  most_similar_paragraph = None
  for i, paragraph_embedding in enumerate(paragraphs_embeddings):
    similarity = cosine_similarity(question_embeddings, paragraph_embedding)
    if similarity > max_similarity:
      max_similarity = similarity
      most_similar_paragraph = i
  return most_similar_paragraph, max_similarity